In [14]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from pickle import dump, load
import os

from skimage.io import imread, imsave
from skimage.transform import resize

classes: 7 - car | 15 - human

### 1: data loader into separate trajectories for each class

In [30]:
all_trajs = {}

In [31]:
all_datadir = '/nfs/diskstation/jren/projections/'
for datadir in os.listdir(all_datadir):
    pickled_path = os.path.join(all_datadir, datadir, 'trajectories.p')
    if not os.path.exists(pickled_path):
        continue
    with open(pickled_path, 'r') as f:
        datas = load(f)
        for data in datas:
            traj_type, traj_timed = data
            if traj_type not in all_trajs:
                all_trajs[traj_type] = []
            traj = [val[0] for val in traj_timed]
            all_trajs[traj_type].append(np.array(traj))

In [38]:
with open('data/all_trajs_raw.p', 'w') as f:
    dump(all_trajs, f)

In [44]:
all_trajs_zeroed = {}
for key, trajs in all_trajs.items():
    if key not in all_trajs_zeroed:
        all_trajs_zeroed[key] = []
    for traj in trajs:
        traj_zero = traj[0]
        all_trajs_zeroed[key].append(traj - traj_zero)

In [51]:
with open('data/all_trajs_zeroed.p', 'w') as f:
    dump(all_trajs_zeroed, f)

In [50]:
all_trajs_deltas = {}
for key, trajs in all_trajs.items():
    if key not in all_trajs_deltas:
        all_trajs_deltas[key] = []
    for traj in trajs:
        traj_delta = traj[1:] - traj[:-1]
        all_trajs_deltas[key].append(traj_delta)

In [52]:
with open('data/all_trajs_deltas.p', 'w') as f:
    dump(all_trajs_deltas, f)

### 2: divide into train and test trajs

In [57]:
train_test_ids = {}
for key, val in all_trajs.items():
    ids = np.arange(len(val))
    np.random.shuffle(ids)
    split = int(0.25 * len(ids))
    train_test_ids[key] = {
        'train': ids[split:], 
        'test': ids[:split]
    }

In [61]:
with open('data/train_test_ids.p', 'w') as f:
    dump(train_test_ids, f)

### 3: Processing Use Sets

In [ ]:
use_sets= {
    'train':set(),
    'test':set()
}

for path in os.listdir('ims/train'):
    use_sets['train'].add(int(path[:-4]))
    
for path in os.listdir('ims/test'):
    use_sets['test'].add(int(path[:-4]))
    
with open('data/use_sets.p', 'w') as f:
    dump(use_sets, f)

### 4: Processing Street Im

In [ ]:
street_raw = imread('/nfs/diskstation/jren/projection_average.png')
street = street_raw[97:719, 510:1132][:,:,:3]
street = resize(street, [1000,1000])
imsave('street.png', street)